In [1]:
PROJECT_ID = "nasa-genie-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
import tempfile
import os
from typing import List, Optional, Tuple, Dict
from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from common.utils.errors import (ResourceNotFoundException,
                                 ValidationError)
from common.utils.http_exceptions import InternalServerError
from utils.errors import NoDocumentsIndexedException
from google.cloud import storage
from services.query import query_prompts
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource
    
from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                        DEFAULT_QUERY_EMBEDDING_MODEL)
from services.query.vertex_search import build_vertex_search

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_deprecated(
/Users/lra

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [10.10.0.2]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 10.10.0.2: (psycopg2.OperationalError) connection to server at "10.10.0.2", port 5432 failed: Network is unreachable
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIE

In [4]:
storage_client = storage.Client(project=PROJECT_ID)

In [5]:
qe_list = QueryEngine.fetch_all()
[(qe.name, qe.id) for qe in qe_list]

[('nasa-search-vertex-v2', '6O2KmKchHgJUSkgrqARq'),
 ('nasa-search-integrated-v5', 'nqIU7Oi4qRIOgKllDqsY'),
 ('nasa-search-shpt-v5', 'bteIzSiaaCd3t8vjzDLw'),
 ('nasa-search-gcs-v5', 'FZnCyfyWNsk2gcHErMVR'),
 ('nasa-search-integrated-v4', 'LjUPZcMrUlo2KU78R8Qf'),
 ('nasa-search-gcs-v4', 'z9mUTW7JehYEvimZlL9p'),
 ('nasa-search-integrated-v3', 'Q6Pwa5VmIL89wSKcwoHQ'),
 ('nasa-search-shpt-v3', 'Qooyhj9PMWmFaiXbuT8E'),
 ('nasa-search-gcs-v3', 'FeY4jc6VCOADwQEXTSej'),
 ('nasa-search-shpt-v2', '4DTQXZfJzLai7tbh3iFX'),
 ('nasa-search-vertex-v1', 'BAlAKbfzfzB33PQQpJLW'),
 ('nasa-search-gcs-v2', 'tgqwj9ANxTY2Sd4iYdrg'),
 ('Nasa Blogs', 'kbWG1DkFPr7xsDUdkxcf'),
 ('nasa-search-integrated-v2', 'TEuxFc0HRTYGYUEmVV1y'),
 ('nasa-search-gcs', '6kcXnzQGxUkWHTcusn5j'),
 ('Vertex Search', 'YNaSypni55kAlF9xp8L8')]

In [7]:
q_engine = QueryEngine.find_by_name('nasa-search-vertex-v2')

QueryDocument.collection.filter(
  "query_engine", "==", 'nasa-search-vertex-v2'
).delete()

QueryDocumentChunk.collection.filter(
  "query_engine_id", "==", q_engine.id
).delete()

QueryReference.collection.filter(
  "query_engine", "==", 'nasa-search-vertex-v2'
).delete()

QueryResult.collection.filter(
  "query_engine", "==", 'nasa-search-vertex-v2'
).delete()
QueryEngine.delete_by_id(q_engine.id)

In [6]:
query_engine = "Test Vertex Search 7"
doc_url = "https://wisconsindot.gov/pages/online-srvcs/external/dmv.aspx"
user_id = "epKVwPZeIBz1rrqcDm1B"
is_public=True
query_description="test vertex search"

In [7]:
from common.models.llm_query import QE_TYPE_VERTEX_SEARCH, QE_TYPE_LLM_SERVICE

# create q_engine
params = {"depth_limit": 1}
query_engine_type = QE_TYPE_VERTEX_SEARCH
embedding_type = DEFAULT_QUERY_EMBEDDING_MODEL
llm_type = DEFAULT_QUERY_EMBEDDING_MODEL
vector_store_type = None
q_engine = QueryEngine(name=query_engine,
                     created_by=user_id,
                     query_engine_type=QE_TYPE_VERTEX_SEARCH,
                     llm_type=llm_type,
                     description=query_description,
                     embedding_type=embedding_type,
                     vector_store=vector_store_type,
                     params=params,
                     doc_url=doc_url)
q_engine.save()

In [9]:
#build_vertex_search(q_engine)

In [10]:
from google.cloud import discoveryengine_v1alpha as discoveryengine
location = "global"
project_id = PROJECT_ID
parent = "projects/lramsey-dev/locations/global/collections/default_collection"

In [12]:
from services.query.vertex_search import (create_data_store, 
                                          create_search_engine, 
                                          import_documents_to_datastore,
                                          download_web_docs,
                                          import_documents_gcs, wait_for_operation, datastore_id_from_engine, inventory_gcs_files,
                                          wait_for_operation)

In [13]:
data_store_id = datastore_id_from_engine(q_engine)
data_store_id

'test-vertex-search-6'

In [14]:
data_url = q_engine.doc_url
project_id = PROJECT_ID
location = "global"

docs_to_be_processed = []
docs_processed = []
docs_not_processed = []

In [15]:
gcs_url, web_docs_downloaded = download_web_docs(q_engine, data_url)

INFO: [query/vertex_search.py:490 - download_web_docs()] downloading web docs to bucket [lramsey-dev-downloads-test-vertex-search-6]
INFO: [utils/gcs_helper.py:45 - create_bucket()] Bucket lramsey-dev-downloads-test-vertex-search-6 created.


2024-02-26 08:18:56 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-02-26 08:18:56 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.12.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.9.13 (v3.9.13:6de2ca5339, May 17 2022, 11:37:23) - [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.2, Platform macOS-14.3.1-x86_64-i386-64bit
2024-02-26 08:18:56 [scrapy.addons] INFO: Enabled addons:
[]
2024-02-26 08:18:56 [py.warnings] WARNING: /Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward comp

INFO: [query/web_datasource.py:46 - save_content()] Saving pages_online-srvcs_external_dmv.aspx.html to /var/folders/65/hxytdjv5109ct635w2lnnj6w00dhhl/T/tmp5gpskqj1
INFO: [query/web_datasource.py:50 - save_content()] 182 bytes written
INFO: [utils/gcs_helper.py:55 - upload_to_gcs()] Uploading pages_online-srvcs_external_dmv.aspx.html to GCS bucket lramsey-dev-downloads-test-vertex-search-6
INFO: [utils/gcs_helper.py:62 - upload_to_gcs()] Uploaded 182 bytes
INFO: [query/web_datasource.py:209 - _item_scraped()] Downloaded Response URL: https://wisconsindot.gov/pages/online-srvcs/external/dmv.aspx
INFO: [query/web_datasource.py:46 - save_content()] Saving Pages_dmv_cons-protect_buy-sell-veh_buyorsellavehicle.aspx.html to /var/folders/65/hxytdjv5109ct635w2lnnj6w00dhhl/T/tmp5gpskqj1
INFO: [query/web_datasource.py:50 - save_content()] 158 bytes written
INFO: [utils/gcs_helper.py:55 - upload_to_gcs()] Uploading Pages_dmv_cons-protect_buy-sell-veh_buyorsellavehicle.aspx.html to GCS bucket lram

2024-02-26 08:19:57 [scrapy.extensions.logstats] INFO: Crawled 111 pages (at 111 pages/min), scraped 104 items (at 104 items/min)


INFO: [query/web_datasource.py:46 - save_content()] Saving .html to /var/folders/65/hxytdjv5109ct635w2lnnj6w00dhhl/T/tmp5gpskqj1
INFO: [query/web_datasource.py:50 - save_content()] 311 bytes written
INFO: [utils/gcs_helper.py:55 - upload_to_gcs()] Uploading .html to GCS bucket lramsey-dev-downloads-test-vertex-search-6
INFO: [utils/gcs_helper.py:62 - upload_to_gcs()] Uploaded 311 bytes
INFO: [query/web_datasource.py:209 - _item_scraped()] Downloaded Response URL: https://mapss.wisconsindot.gov/
INFO: [query/web_datasource.py:46 - save_content()] Saving Pages_about-wisdot_who-we-are_dsp_default.aspx.html to /var/folders/65/hxytdjv5109ct635w2lnnj6w00dhhl/T/tmp5gpskqj1
INFO: [query/web_datasource.py:50 - save_content()] 233 bytes written
INFO: [utils/gcs_helper.py:55 - upload_to_gcs()] Uploading Pages_about-wisdot_who-we-are_dsp_default.aspx.html to GCS bucket lramsey-dev-downloads-test-vertex-search-6
INFO: [utils/gcs_helper.py:62 - upload_to_gcs()] Uploaded 233 bytes
INFO: [query/web_da

2024-02-26 08:20:50 [scrapy.core.engine] INFO: Closing spider (finished)
2024-02-26 08:20:50 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 113862,
 'downloader/request_count': 244,
 'downloader/request_method_count/GET': 244,
 'downloader/response_bytes': 37360370,
 'downloader/response_count': 244,
 'downloader/response_status_count/200': 244,
 'elapsed_time_seconds': 113.732351,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2024, 2, 26, 13, 20, 50, 583498, tzinfo=datetime.timezone.utc),
 'item_scraped_count': 243,
 'log_count/INFO': 11,
 'log_count/WARNING': 1,
 'memusage/max': 506593280,
 'memusage/startup': 418586624,
 'request_depth_max': 1,
 'response_received_count': 244,
 'scheduler/dequeued': 244,
 'scheduler/dequeued/memory': 244,
 'scheduler/enqueued': 244,
 'scheduler/enqueued/memory': 244,
 'start_time': datetime.datetime(2024, 2, 26, 13, 18, 56, 851147, tzinfo=datetime.timezone.utc)}
2024-02-26 08:20:50 [scrapy.core.en

finished
https://wisconsindot.gov/pages/online-srvcs/external/dmv.aspx
https://wisconsindot.gov/Pages/dmv/cons-protect/buy-sell-veh/buyorsellavehicle.aspx
https://wisconsindot.gov/Pages/dmv/cons-protect/lemon-law/lemonlaw.aspx
https://wisconsindot.gov/Pages/dmv/agri-eq-veh/ioh.aspx
https://wisconsindot.gov/Pages/dmv/agri-eq-veh/ag-cmv.aspx
https://wisconsindot.gov/Pages/dmv/cons-protect/know-rghts/knowyourrights.aspx
https://wisconsindot.gov/Pages/dmv/agri-eq-veh/axle-weights.aspx
https://wisconsindot.gov/Pages/dmv/cons-protect/file-cmplnt/filedealercomplaint.aspx
https://wisconsindot.gov/Pages/dmv/agri-eq-veh/default.aspx
https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-training/default.aspx
https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-inds-plt/dlrspecialplate.aspx
https://wisconsindot.gov/Pages/dmv/dlr-agents/tips-tls-dlr/tips.aspx
https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-frms-pbs/dealer-forms.aspx
https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-ttl-reg/default.aspx


In [16]:
web_docs_downloaded

['https://wisconsindot.gov/pages/online-srvcs/external/dmv.aspx',
 'https://wisconsindot.gov/Pages/dmv/cons-protect/buy-sell-veh/buyorsellavehicle.aspx',
 'https://wisconsindot.gov/Pages/dmv/cons-protect/lemon-law/lemonlaw.aspx',
 'https://wisconsindot.gov/Pages/dmv/agri-eq-veh/ioh.aspx',
 'https://wisconsindot.gov/Pages/dmv/agri-eq-veh/ag-cmv.aspx',
 'https://wisconsindot.gov/Pages/dmv/cons-protect/know-rghts/knowyourrights.aspx',
 'https://wisconsindot.gov/Pages/dmv/agri-eq-veh/axle-weights.aspx',
 'https://wisconsindot.gov/Pages/dmv/cons-protect/file-cmplnt/filedealercomplaint.aspx',
 'https://wisconsindot.gov/Pages/dmv/agri-eq-veh/default.aspx',
 'https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-training/default.aspx',
 'https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-inds-plt/dlrspecialplate.aspx',
 'https://wisconsindot.gov/Pages/dmv/dlr-agents/tips-tls-dlr/tips.aspx',
 'https://wisconsindot.gov/Pages/dmv/dlr-agents/dlr-frms-pbs/dealer-forms.aspx',
 'https://wisconsindot.gov/P

In [18]:
gcs_url

'gs://lramsey-dev-downloads-test-vertex-search-6'

In [19]:
docs_to_be_processed = inventory_gcs_files(gcs_url)

In [20]:
docs_to_be_processed

['gs://lramsey-dev-downloads-test-vertex-search-6/Pages_Home.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_DivEqInc.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_accessibility-statement.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_careers_benefits.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_careers_default.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_careers_employees.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_careers_equal.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_contact-us_default.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_contact-us_dmv-contact.aspx.html',
 'gs://lramsey-dev-downloads-test-vertex-search-6/Pages_about-wisdot_contact-us_hill-farms-bldg.aspx.html',
 'gs://lramsey-dev-downloads-te

In [ ]:
operation = create_data_store(q_engine, project_id, data_store_id)
wait_for_operation(operation)

In [ ]:
client = discoveryengine.DocumentServiceClient()
parent = client.branch_path(
    project=project_id,
    location=location,
    data_store=data_store_id,
    branch="default_branch",
)

In [ ]:
data_url

In [ ]:
operation = import_documents_gcs(data_url,
                                 docs_to_be_processed,
                                 client,
                                 parent)

In [ ]:
print(f"Waiting for import operation to complete: {operation.operation.name}")

In [ ]:
wait_for_operation(operation)

In [ ]:
metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

In [ ]:
metadata

In [ ]:
operation.__dict__

In [ ]:
operation.result()

In [ ]:
!gsutil ls -R gs://944045413892_us_import_content/errors16429642499957673123

In [ ]:
if metadata.success_count == len(docs_to_be_processed):
  docs_processed = docs_to_be_processed
else:
  # TODO: build list of documents processed/not processed from results
  pass

In [ ]:
docs_processed

In [ ]:
operation = create_search_engine(q_engine, project_id, data_store_id)

In [ ]:
q_engine.index_id = data_store_id
q_engine.update()

In [ ]:
from services.query.vertex_search import query_vertex_search, perform_vertex_search

In [ ]:
search_query = "what qualifies as an encounter in medicaid billing?"

In [ ]:
client = discoveryengine.SearchServiceClient()

In [ ]:
serving_config = client.serving_config_path(
    project=project_id,
    location=location,
    data_store=data_store_id,
    serving_config="default_config",
)

In [ ]:
content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
    snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
        return_snippet=True
    ),
    summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
        summary_result_count=5,
        include_citations=True,
        ignore_adversarial_query=True,
        ignore_non_summary_seeking_query=True,
    ),
)

In [ ]:
request = discoveryengine.SearchRequest(
    serving_config=serving_config,
    query=search_query,
    page_size=10,
    content_search_spec=content_search_spec,
    query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
        condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
    ),
    spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
        mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
    ),
)

In [ ]:
response = client.search(request)

In [ ]:
len(response.results)

In [ ]:
results = response.results
type(results[0])

In [ ]:
type(response)

In [ ]:
document = results[0].document

In [ ]:
type(document)

In [ ]:
from google.protobuf.json_format import MessageToDict

In [ ]:
import proto

In [ ]:
document_dict = proto.Message.to_dict(document)

In [ ]:
document_dict.keys()

In [ ]:
document_dict["derived_struct_data"]["snippets"][0]["snippet"]

In [ ]:
document_dict["derived_struct_data"]

In [ ]:
document_data = proto.Message.to_dict(document)["derived_struct_data"]

In [ ]:
document_data["link"]

In [ ]:
document_data["snippets"][0]["snippet"]